In [1]:
import pandas as pd

# Load observations
obs = pd.read_csv("/content/observations.csv")

# Rename columns for consistency
obs = obs.rename(columns={
    "DATE": "OBS_DATE",
    "PATIENT": "PATIENT_ID",
    "ENCOUNTER": "ENCOUNTER_ID",
    "CODE": "OBS_CODE",
    "DESCRIPTION": "OBS_DESC",
    "VALUE": "OBS_VALUE",
    "UNITS": "OBS_UNITS",
    "TYPE": "OBS_TYPE"
})

# Convert OBS_DATE to datetime
obs["OBS_DATE"] = pd.to_datetime(obs["OBS_DATE"], errors="coerce")

# Convert VALUE to numeric; non-numeric -> NaN
obs["OBS_VALUE_NUM"] = pd.to_numeric(obs["OBS_VALUE"], errors="coerce")

# Optional: drop rows where OBS_DATE or PATIENT_ID is missing
obs = obs.dropna(subset=["OBS_DATE", "PATIENT_ID"])

# Check basic stats
print("Number of observations:", len(obs))
print("Number of numeric observations:", obs["OBS_VALUE_NUM"].notna().sum())
print(obs.head())


/tmp/ipython-input-243714448.py:4: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  obs = pd.read_csv("/content/observations.csv")


Number of observations: 365881
Number of numeric observations: 365881
                   OBS_DATE                            PATIENT_ID  \
0 2015-04-15 17:34:48+00:00  2932ce25-3a51-adcf-de0a-dd123262649a   
1 2015-05-22 15:26:56+00:00  66ed04f7-0ab2-dfab-cfb9-06bc321dbcd4   
2 2015-04-30 17:38:34+00:00  07fea930-1bfe-3e6f-dbe6-0277b8ff45ac   
3 2015-11-12 15:51:39+00:00  56dec753-c765-e129-93bd-8592bcca4614   
4 2016-04-15 17:34:48+00:00  2932ce25-3a51-adcf-de0a-dd123262649a   

  ENCOUNTER_ID CATEGORY OBS_CODE OBS_DESC  OBS_VALUE OBS_UNITS OBS_TYPE  \
0          NaN      NaN     QALY     QALY       21.0         a  numeric   
1          NaN      NaN     QALY     QALY       18.0         a  numeric   
2          NaN      NaN     QALY     QALY       24.0         a  numeric   
3          NaN      NaN     QALY     QALY       12.0         a  numeric   
4          NaN      NaN     QALY     QALY       22.0         a  numeric   

   OBS_VALUE_NUM  
0           21.0  
1           18.0  
2      

In [2]:
# ---------- 1) Keep only numeric observations ----------
obs_num = obs.dropna(subset=["OBS_VALUE_NUM"])

# ---------- 2) Aggregate per patient, per day, per observation code ----------
obs_daily = (
    obs_num.groupby(["PATIENT_ID", "OBS_DATE", "OBS_CODE"], as_index=False)
           .agg({"OBS_VALUE_NUM": "mean"})
)

# ---------- 3) Pivot to wide format (each OBS_CODE is a column) ----------
obs_daily_wide = obs_daily.pivot_table(
    index=["PATIENT_ID", "OBS_DATE"],
    columns="OBS_CODE",
    values="OBS_VALUE_NUM"
)

# ---------- 4) Reset index ----------
obs_daily_wide = obs_daily_wide.reset_index()

# ---------- 5) Drop columns where >90% of values are NaN ----------
threshold = 0.9
cols_to_drop = obs_daily_wide.columns[(obs_daily_wide.isna().mean() > threshold)]
obs_daily_wide = obs_daily_wide.drop(columns=cols_to_drop)

# ---------- 6) Forward-fill missing values per patient ----------
# Sort first
obs_daily_wide = obs_daily_wide.sort_values(["PATIENT_ID", "OBS_DATE"])

# Forward-fill per patient, limit 7 days
obs_daily_wide_ffill = obs_daily_wide.groupby("PATIENT_ID").ffill(limit=7)

# Merge back PATIENT_ID and OBS_DATE columns (they were not NaN)
obs_daily_wide_ffill["PATIENT_ID"] = obs_daily_wide["PATIENT_ID"]
obs_daily_wide_ffill["OBS_DATE"] = obs_daily_wide["OBS_DATE"]

# ---------- 7) Keep only rows where at least one numeric observation exists ----------
obs_daily_wide_ffill = obs_daily_wide.groupby("PATIENT_ID", group_keys=False).apply(lambda x: x.ffill(limit=7))

# ---------- 8) Check result ----------
print(obs_daily_wide_ffill.head())
print("Shape after cleaning:", obs_daily_wide_ffill.shape)


OBS_CODE                            PATIENT_ID                  OBS_DATE  \
0         00092f43-1aee-3c4e-3160-58b40a45fb2b 2015-09-03 10:00:37+00:00   
1         00092f43-1aee-3c4e-3160-58b40a45fb2b 2016-09-03 10:00:37+00:00   
2         00092f43-1aee-3c4e-3160-58b40a45fb2b 2017-09-03 10:00:37+00:00   
3         00092f43-1aee-3c4e-3160-58b40a45fb2b 2018-09-03 10:00:37+00:00   
4         00092f43-1aee-3c4e-3160-58b40a45fb2b 2019-09-03 10:00:37+00:00   

OBS_CODE  DALY  QALY  QOLS  
0          0.0  10.0   1.0  
1          0.0  11.0   1.0  
2          0.0  12.0   1.0  
3          0.0  13.0   1.0  
4          0.0  14.0   1.0  
Shape after cleaning: (122121, 5)


/tmp/ipython-input-2548534800.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  obs_daily_wide_ffill = obs_daily_wide.groupby("PATIENT_ID", group_keys=False).apply(lambda x: x.ffill(limit=7))


In [3]:
obs_daily

,PATIENT_ID,OBS_DATE,OBS_CODE,OBS_VALUE_NUM
0,00092f43-1aee-3c4e-3160-58b40a45fb2b,2015-09-03 10:00:37+00:00,DALY,0.0
1,00092f43-1aee-3c4e-3160-58b40a45fb2b,2015-09-03 10:00:37+00:00,QALY,10.0
2,00092f43-1aee-3c4e-3160-58b40a45fb2b,2015-09-03 10:00:37+00:00,QOLS,1.0
3,00092f43-1aee-3c4e-3160-58b40a45fb2b,2016-09-03 10:00:37+00:00,DALY,0.0
4,00092f43-1aee-3c4e-3160-58b40a45fb2b,2016-09-03 10:00:37+00:00,QALY,11.0
...,...,...,...,...
365876,fffc034a-0a09-cb96-1779-0953de2da5e0,2024-04-23 17:16:20+00:00,QALY,60.0
365877,fffc034a-0a09-cb96-1779-0953de2da5e0,2024-04-23 17:16:20+00:00,QOLS,1.0
365878,fffc034a-0a09-cb96-1779-0953de2da5e0,2025-04-23 17:16:20+00:00,DALY,0.0
365879,fffc034a-0a09-cb96-1779-0953de2da5e0,2025-04-23 17:16:20+00:00,QALY,61.0


In [4]:
obs_daily_wide_ffill

OBS_CODE,PATIENT_ID,OBS_DATE,DALY,QALY,QOLS
0,00092f43-1aee-3c4e-3160-58b40a45fb2b,2015-09-03 10:00:37+00:00,0.0,10.0,1.0
1,00092f43-1aee-3c4e-3160-58b40a45fb2b,2016-09-03 10:00:37+00:00,0.0,11.0,1.0
2,00092f43-1aee-3c4e-3160-58b40a45fb2b,2017-09-03 10:00:37+00:00,0.0,12.0,1.0
3,00092f43-1aee-3c4e-3160-58b40a45fb2b,2018-09-03 10:00:37+00:00,0.0,13.0,1.0
4,00092f43-1aee-3c4e-3160-58b40a45fb2b,2019-09-03 10:00:37+00:00,0.0,14.0,1.0
...,...,...,...,...,...
122116,fffc034a-0a09-cb96-1779-0953de2da5e0,2021-04-23 17:16:20+00:00,0.0,57.0,1.0
122117,fffc034a-0a09-cb96-1779-0953de2da5e0,2022-04-23 17:16:20+00:00,0.0,58.0,1.0
122118,fffc034a-0a09-cb96-1779-0953de2da5e0,2023-04-23 17:16:20+00:00,0.0,59.0,1.0
122119,fffc034a-0a09-cb96-1779-0953de2da5e0,2024-04-23 17:16:20+00:00,0.0,60.0,1.0


In [5]:
import pandas as pd
enc = pd.read_csv("/content/encounters.csv")

# Keep only needed columns and rename
enc_clean = enc.rename(columns={
    "Id": "ENCOUNTER_ID",
    "START": "ENCOUNTER_START",
    "STOP": "ENCOUNTER_STOP",
    "PATIENT": "PATIENT_ID",
    "ENCOUNTERCLASS": "ENCOUNTER_CLASS"
})[["ENCOUNTER_ID", "PATIENT_ID", "ENCOUNTER_START", "ENCOUNTER_STOP", "ENCOUNTER_CLASS"]]

# Convert to datetime
enc_clean["ENCOUNTER_START"] = pd.to_datetime(enc_clean["ENCOUNTER_START"], utc=True, errors="coerce")
enc_clean["ENCOUNTER_STOP"] = pd.to_datetime(enc_clean["ENCOUNTER_STOP"], utc=True, errors="coerce")

# Sort by patient and encounter start
enc_clean = enc_clean.sort_values(["PATIENT_ID", "ENCOUNTER_START"])

# Check first few rows
enc_clean.head()


,ENCOUNTER_ID,PATIENT_ID,ENCOUNTER_START,ENCOUNTER_STOP,ENCOUNTER_CLASS
41428,5fc53219-f1af-ced0-2051-d8f56d254b81,00092f43-1aee-3c4e-3160-58b40a45fb2b,2015-09-18 10:00:37+00:00,2015-09-18 10:15:37+00:00,wellness
41439,344cb013-ffe4-304f-0ebe-bd1f7db5f848,00092f43-1aee-3c4e-3160-58b40a45fb2b,2016-09-23 10:00:37+00:00,2016-09-23 10:15:37+00:00,wellness
41446,fbc349d1-608b-643c-88b9-a2e9fb053e39,00092f43-1aee-3c4e-3160-58b40a45fb2b,2017-09-29 10:00:37+00:00,2017-09-29 10:15:37+00:00,wellness
41447,a69237e5-7d1d-4a01-8fa3-6ef84ff1a5f4,00092f43-1aee-3c4e-3160-58b40a45fb2b,2018-10-05 10:00:37+00:00,2018-10-05 10:15:37+00:00,wellness
41448,90b39b36-d5c0-8455-aa40-5c8381744fd5,00092f43-1aee-3c4e-3160-58b40a45fb2b,2019-10-11 10:00:37+00:00,2019-10-11 10:15:37+00:00,wellness


In [6]:
enc_clean

,ENCOUNTER_ID,PATIENT_ID,ENCOUNTER_START,ENCOUNTER_STOP,ENCOUNTER_CLASS
41428,5fc53219-f1af-ced0-2051-d8f56d254b81,00092f43-1aee-3c4e-3160-58b40a45fb2b,2015-09-18 10:00:37+00:00,2015-09-18 10:15:37+00:00,wellness
41439,344cb013-ffe4-304f-0ebe-bd1f7db5f848,00092f43-1aee-3c4e-3160-58b40a45fb2b,2016-09-23 10:00:37+00:00,2016-09-23 10:15:37+00:00,wellness
41446,fbc349d1-608b-643c-88b9-a2e9fb053e39,00092f43-1aee-3c4e-3160-58b40a45fb2b,2017-09-29 10:00:37+00:00,2017-09-29 10:15:37+00:00,wellness
41447,a69237e5-7d1d-4a01-8fa3-6ef84ff1a5f4,00092f43-1aee-3c4e-3160-58b40a45fb2b,2018-10-05 10:00:37+00:00,2018-10-05 10:15:37+00:00,wellness
41448,90b39b36-d5c0-8455-aa40-5c8381744fd5,00092f43-1aee-3c4e-3160-58b40a45fb2b,2019-10-11 10:00:37+00:00,2019-10-11 10:15:37+00:00,wellness
...,...,...,...,...,...
97230,4d99ab11-d1cd-444b-56f5-94e66ddb601c,fffc034a-0a09-cb96-1779-0953de2da5e0,2021-12-21 17:16:20+00:00,2021-12-21 17:31:20+00:00,outpatient
97231,c06cbb03-f0bc-9c93-f0e5-d9a2be115424,fffc034a-0a09-cb96-1779-0953de2da5e0,2022-06-14 17:16:20+00:00,2022-06-14 17:31:20+00:00,wellness
97232,616bc3df-bbe1-6872-9036-2f4f110fbee3,fffc034a-0a09-cb96-1779-0953de2da5e0,2023-06-20 17:16:20+00:00,2023-06-20 17:31:20+00:00,wellness
97233,4d02d34a-f90d-feac-a85f-f4fc1cae778b,fffc034a-0a09-cb96-1779-0953de2da5e0,2024-06-25 17:16:20+00:00,2024-06-25 17:31:20+00:00,wellness


In [7]:
import pandas as pd

patients = pd.read_csv("/content/patients.csv")

# ---------- 1) Keep only relevant patient columns ----------
# Select columns that are meaningful for prediction
patient_cols = [
    "Id", "BIRTHDATE", "DEATHDATE", "GENDER", "RACE", "ETHNICITY",
    "INCOME", "HEALTHCARE_COVERAGE", "HEALTHCARE_EXPENSES"
]
patients_subset = patients[patient_cols].copy()

# Rename Id to PATIENT_ID for consistency
patients_subset = patients_subset.rename(columns={"Id": "PATIENT_ID"})

# ---------- 2) Parse date columns ----------
patients_subset["BIRTHDATE"] = pd.to_datetime(patients_subset["BIRTHDATE"], utc=True, errors="coerce")
patients_subset["DEATHDATE"] = pd.to_datetime(patients_subset["DEATHDATE"], utc=True, errors="coerce")

# ---------- 3) Compute patient age at each observation ----------
# Merge demographics with daily observations
obs_patient = obs_daily_wide_ffill.merge(
    patients_subset,
    on="PATIENT_ID",
    how="left"
)

# Compute age in years at the date of observation
obs_patient["AGE"] = (obs_patient["OBS_DATE"] - obs_patient["BIRTHDATE"]).dt.days / 365.25

# ---------- 4) Optional: drop unnecessary columns ----------
# Keep only useful static features
static_cols = ["PATIENT_ID", "OBS_DATE", "AGE", "GENDER", "RACE", "ETHNICITY",
               "INCOME", "HEALTHCARE_COVERAGE", "HEALTHCARE_EXPENSES"]

# Dynamic observation columns
dynamic_cols = obs_daily_wide_ffill.columns.difference(["PATIENT_ID", "OBS_DATE"])

# Final patient-day table
patient_day_table = obs_patient[["PATIENT_ID", "OBS_DATE"] + dynamic_cols.tolist() + static_cols[2:]]

# ---------- 5) Check results ----------
print(patient_day_table.head())
print("Shape of patient-day table:", patient_day_table.shape)

                             PATIENT_ID                  OBS_DATE  DALY  QALY  \
0  00092f43-1aee-3c4e-3160-58b40a45fb2b 2015-09-03 10:00:37+00:00   0.0  10.0   
1  00092f43-1aee-3c4e-3160-58b40a45fb2b 2016-09-03 10:00:37+00:00   0.0  11.0   
2  00092f43-1aee-3c4e-3160-58b40a45fb2b 2017-09-03 10:00:37+00:00   0.0  12.0   
3  00092f43-1aee-3c4e-3160-58b40a45fb2b 2018-09-03 10:00:37+00:00   0.0  13.0   
4  00092f43-1aee-3c4e-3160-58b40a45fb2b 2019-09-03 10:00:37+00:00   0.0  14.0   

   QOLS        AGE GENDER   RACE    ETHNICITY  INCOME  HEALTHCARE_COVERAGE  \
0   1.0  10.997947      M  white  nonhispanic  139443                  0.0   
1   1.0  12.000000      M  white  nonhispanic  139443                  0.0   
2   1.0  12.999316      M  white  nonhispanic  139443                  0.0   
3   1.0  13.998631      M  white  nonhispanic  139443                  0.0   
4   1.0  14.997947      M  white  nonhispanic  139443                  0.0   

   HEALTHCARE_EXPENSES  
0              1035

In [8]:
# ---------- 1) Identify categorical columns ----------
categorical_cols = ["GENDER", "RACE", "ETHNICITY"]

# ---------- 2) One-hot encode categorical columns ----------
patient_day_encoded = pd.get_dummies(
    patient_day_table,
    columns=categorical_cols,
    drop_first=False  # keep all levels
)

# ---------- 3) Check the first few rows ----------
print(patient_day_encoded.head())
print("Shape after encoding:", patient_day_encoded.shape)

# ---------- 4) Verify dummy columns ----------
print("New columns added from categorical features:")
print([col for col in patient_day_encoded.columns if any(x in col for x in categorical_cols)])


                             PATIENT_ID                  OBS_DATE  DALY  QALY  \
0  00092f43-1aee-3c4e-3160-58b40a45fb2b 2015-09-03 10:00:37+00:00   0.0  10.0   
1  00092f43-1aee-3c4e-3160-58b40a45fb2b 2016-09-03 10:00:37+00:00   0.0  11.0   
2  00092f43-1aee-3c4e-3160-58b40a45fb2b 2017-09-03 10:00:37+00:00   0.0  12.0   
3  00092f43-1aee-3c4e-3160-58b40a45fb2b 2018-09-03 10:00:37+00:00   0.0  13.0   
4  00092f43-1aee-3c4e-3160-58b40a45fb2b 2019-09-03 10:00:37+00:00   0.0  14.0   

   QOLS        AGE  INCOME  HEALTHCARE_COVERAGE  HEALTHCARE_EXPENSES  \
0   1.0  10.997947  139443                  0.0              10358.4   
1   1.0  12.000000  139443                  0.0              10358.4   
2   1.0  12.999316  139443                  0.0              10358.4   
3   1.0  13.998631  139443                  0.0              10358.4   
4   1.0  14.997947  139443                  0.0              10358.4   

   GENDER_F  GENDER_M  RACE_asian  RACE_black  RACE_hawaiian  RACE_native  \
0  

In [9]:
import pandas as pd

med = pd.read_csv("/content/medications.csv")

# ---------- 1) Convert START and STOP to datetime ----------
med["START"] = pd.to_datetime(med["START"])
med["STOP"] = pd.to_datetime(med["STOP"])

# For missing STOP, assume same day
med["STOP"] = med["STOP"].fillna(med["START"])

# ---------- 2) Expand each medication to daily rows ----------
med_expanded = []

for idx, row in med.iterrows():
    patient_id = row["PATIENT"]
    start = row["START"]
    stop = row["STOP"]
    med_code = row["CODE"]

    # Generate all dates from START to STOP
    date_range = pd.date_range(start=start, end=stop)

    for dt in date_range:
        med_expanded.append([patient_id, dt, med_code])

med_expanded = pd.DataFrame(med_expanded, columns=["PATIENT_ID", "DATE", "MED_CODE"])

# ---------- 3) Pivot to wide format (binary flags) ----------
med_daily_wide = med_expanded.pivot_table(
    index=["PATIENT_ID", "DATE"],
    columns="MED_CODE",
    aggfunc=lambda x: 1,  # flag = 1 if medication exists
    fill_value=0          # fill remaining NaNs with 0
)

# Flatten column MultiIndex
med_daily_wide.columns = med_daily_wide.columns.map(str)

# Reset index
med_daily_wide = med_daily_wide.reset_index()

# ---------- 4) Drop sparse columns (>90% NaN originally) ----------
# Note: fill_value=0 ensures no NaNs now, so check original column frequency
med_counts = med_expanded.groupby("MED_CODE")["PATIENT_ID"].nunique() / med["PATIENT"].nunique()
sparse_meds = med_counts[med_counts < 0.1].index  # meds in <10% of patients
med_daily_wide = med_daily_wide.drop(columns=sparse_meds.astype(str))

# ---------- 5) Check result ----------
print(med_daily_wide.head())
print("Shape after processing medications:", med_daily_wide.shape)


MED_CODE                            PATIENT_ID                      DATE  \
0         009b5ce5-cf37-69b4-df69-0fd6576f7a90 2021-10-14 09:33:36+00:00   
1         009b5ce5-cf37-69b4-df69-0fd6576f7a90 2021-11-13 09:33:36+00:00   
2         00c0966e-93fd-8c23-cb02-8463c0436b92 2009-11-24 06:48:38+00:00   
3         00c0966e-93fd-8c23-cb02-8463c0436b92 2009-12-24 06:48:38+00:00   
4         010710b1-3f72-2ebb-47b7-84aef0d7303e 2018-01-28 16:42:39+00:00   

MED_CODE  308136  310798  314076  
0              0       0       1  
1              0       1       0  
2              1       0       0  
3              0       0       1  
4              1       0       0  
Shape after processing medications: (47333, 5)


In [10]:
import pandas as pd

# ---------- 1) Load and convert START/STOP ----------
con = pd.read_csv("/content/conditions.csv")

con["START"] = pd.to_datetime(con["START"])
con["STOP"] = pd.to_datetime(con["STOP"])

# For missing STOP, assume 365-day duration
con["STOP"] = con["STOP"].fillna(con["START"] + pd.Timedelta(days=365))

# ---------- 2) Expand each condition to daily rows ----------
con_expanded = []

for idx, row in con.iterrows():
    patient_id = row["PATIENT"]
    start = row["START"]
    stop = row["STOP"]
    con_code = row["CODE"]

    # Generate all dates from START to STOP
    date_range = pd.date_range(start=start, end=stop)

    for dt in date_range:
        con_expanded.append([patient_id, dt, con_code])

con_expanded = pd.DataFrame(con_expanded, columns=["PATIENT_ID", "DATE", "CON_CODE"])

# ---------- 3) Pivot to wide format (binary flags) ----------
con_daily_wide = con_expanded.pivot_table(
    index=["PATIENT_ID", "DATE"],
    columns="CON_CODE",
    aggfunc=lambda x: 1,  # flag = 1 if condition exists
    fill_value=0          # fill remaining NaNs with 0
)

# Flatten column MultiIndex
con_daily_wide.columns = con_daily_wide.columns.map(str)

# Reset index
con_daily_wide = con_daily_wide.reset_index()

# ---------- 4) Drop sparse conditions (<10% of patients) ----------
con_counts = con_expanded.groupby("CON_CODE")["PATIENT_ID"].nunique() / con["PATIENT"].nunique()
sparse_cons = con_counts[con_counts < 0.1].index  # conditions in <10% of patients
con_daily_wide = con_daily_wide.drop(columns=sparse_cons.astype(str))

# ---------- 5) Check result ----------
print(con_daily_wide.head())
print("Shape after processing conditions:", con_daily_wide.shape)


CON_CODE                            PATIENT_ID       DATE  59621000
0         009b5ce5-cf37-69b4-df69-0fd6576f7a90 2021-10-14         1
1         009b5ce5-cf37-69b4-df69-0fd6576f7a90 2021-10-15         1
2         009b5ce5-cf37-69b4-df69-0fd6576f7a90 2021-10-16         1
3         009b5ce5-cf37-69b4-df69-0fd6576f7a90 2021-10-17         1
4         009b5ce5-cf37-69b4-df69-0fd6576f7a90 2021-10-18         1
Shape after processing conditions: (255756, 3)


In [11]:
# ---------- 4.1 Merge obs and med ----------

# Ensure medication flags are filled with 0 where NaN
med_daily_wide_filled = med_daily_wide.fillna(0)

# Merge obs_daily_wide_ffill with med_daily_wide_filled
patient_day_df = pd.merge(
    obs_daily_wide_ffill,
    med_daily_wide_filled,
    how='outer',
    left_on=['PATIENT_ID', 'OBS_DATE'],
    right_on=['PATIENT_ID', 'DATE']
)

# Drop the redundant 'DATE' column from the merge
patient_day_df = patient_day_df.drop(columns=['DATE'])

# Check intermediate result
print("Shape after merging obs + med:", patient_day_df.shape)
print(patient_day_df.head())

Shape after merging obs + med: (169294, 8)
                             PATIENT_ID                  OBS_DATE  DALY  QALY  \
0  00092f43-1aee-3c4e-3160-58b40a45fb2b 2015-09-03 10:00:37+00:00   0.0  10.0   
1  00092f43-1aee-3c4e-3160-58b40a45fb2b 2016-09-03 10:00:37+00:00   0.0  11.0   
2  00092f43-1aee-3c4e-3160-58b40a45fb2b 2017-09-03 10:00:37+00:00   0.0  12.0   
3  00092f43-1aee-3c4e-3160-58b40a45fb2b 2018-09-03 10:00:37+00:00   0.0  13.0   
4  00092f43-1aee-3c4e-3160-58b40a45fb2b 2019-09-03 10:00:37+00:00   0.0  14.0   

   QOLS  308136  310798  314076  
0   1.0     NaN     NaN     NaN  
1   1.0     NaN     NaN     NaN  
2   1.0     NaN     NaN     NaN  
3   1.0     NaN     NaN     NaN  
4   1.0     NaN     NaN     NaN  


In [12]:
# ---------- 4.2 Merge with conditions ----------

# Fill NaNs in conditions with 0
con_daily_wide_filled = con_daily_wide.fillna(0)

# Convert the 'DATE' column in con_daily_wide_filled to UTC to match patient_day_df['OBS_DATE']
con_daily_wide_filled['DATE'] = pd.to_datetime(con_daily_wide_filled['DATE'], utc=True)


# Merge with existing patient_day_df
patient_day_df = pd.merge(
    patient_day_df,
    con_daily_wide_filled,
    how='outer',
    left_on=['PATIENT_ID', 'OBS_DATE'],
    right_on=['PATIENT_ID', 'DATE']
)

# Drop the redundant 'DATE' column from the merge
patient_day_df = patient_day_df.drop(columns=['DATE'])


# Check final merged result
print("Shape after merging obs + med + con:", patient_day_df.shape)
print(patient_day_df.head())

Shape after merging obs + med + con: (425050, 9)
                             PATIENT_ID                  OBS_DATE  DALY  QALY  \
0  00092f43-1aee-3c4e-3160-58b40a45fb2b 2015-09-03 10:00:37+00:00   0.0  10.0   
1  00092f43-1aee-3c4e-3160-58b40a45fb2b 2016-09-03 10:00:37+00:00   0.0  11.0   
2  00092f43-1aee-3c4e-3160-58b40a45fb2b 2017-09-03 10:00:37+00:00   0.0  12.0   
3  00092f43-1aee-3c4e-3160-58b40a45fb2b 2018-09-03 10:00:37+00:00   0.0  13.0   
4  00092f43-1aee-3c4e-3160-58b40a45fb2b 2019-09-03 10:00:37+00:00   0.0  14.0   

   QOLS  308136  310798  314076  59621000  
0   1.0     NaN     NaN     NaN       NaN  
1   1.0     NaN     NaN     NaN       NaN  
2   1.0     NaN     NaN     NaN       NaN  
3   1.0     NaN     NaN     NaN       NaN  
4   1.0     NaN     NaN     NaN       NaN  


In [13]:
patient_day_df

,PATIENT_ID,OBS_DATE,DALY,QALY,QOLS,308136,310798,314076,59621000
0,00092f43-1aee-3c4e-3160-58b40a45fb2b,2015-09-03 10:00:37+00:00,0.0,10.0,1.0,NaN,NaN,NaN,NaN
1,00092f43-1aee-3c4e-3160-58b40a45fb2b,2016-09-03 10:00:37+00:00,0.0,11.0,1.0,NaN,NaN,NaN,NaN
2,00092f43-1aee-3c4e-3160-58b40a45fb2b,2017-09-03 10:00:37+00:00,0.0,12.0,1.0,NaN,NaN,NaN,NaN
3,00092f43-1aee-3c4e-3160-58b40a45fb2b,2018-09-03 10:00:37+00:00,0.0,13.0,1.0,NaN,NaN,NaN,NaN
4,00092f43-1aee-3c4e-3160-58b40a45fb2b,2019-09-03 10:00:37+00:00,0.0,14.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
425045,fffc034a-0a09-cb96-1779-0953de2da5e0,2021-04-23 17:16:20+00:00,0.0,57.0,1.0,NaN,NaN,NaN,NaN
425046,fffc034a-0a09-cb96-1779-0953de2da5e0,2022-04-23 17:16:20+00:00,0.0,58.0,1.0,NaN,NaN,NaN,NaN
425047,fffc034a-0a09-cb96-1779-0953de2da5e0,2023-04-23 17:16:20+00:00,0.0,59.0,1.0,NaN,NaN,NaN,NaN
425048,fffc034a-0a09-cb96-1779-0953de2da5e0,2024-04-23 17:16:20+00:00,0.0,60.0,1.0,NaN,NaN,NaN,NaN


In [14]:
# ---------- 4.3 Sort by patient and date ----------
patient_day_df = patient_day_df.sort_values(["PATIENT_ID", "OBS_DATE"]).reset_index(drop=True)

# ---------- 4.4 Optional: Verify columns ----------
print("Columns in final dataset:", patient_day_df.columns.tolist())


Columns in final dataset: ['PATIENT_ID', 'OBS_DATE', 'DALY', 'QALY', 'QOLS', '308136', '310798', '314076', '59621000']


In [15]:
patient_day_df['308136'].isna().value_counts()

,count
308136,
True,377717
False,47333


In [16]:
patient_day_df['310798'].isna().value_counts()

,count
310798,
True,377717
False,47333


In [17]:
patient_day_df['314076'].isna().value_counts()

,count
314076,
True,377717
False,47333


In [18]:
patient_day_df['59621000'].isna().value_counts()

,count
59621000,
False,255756
True,169294


In [19]:
# Cell A: normalize columns & datetimes
import pandas as pd

# Make sure patient_day_df has OBS_DATE as datetime and consistent timezone
patient_day_df["OBS_DATE"] = pd.to_datetime(patient_day_df["OBS_DATE"], utc=True, errors="coerce")

# Prepare enc_clean: ensure datetime and required columns exist
enc_clean["ENCOUNTER_START"] = pd.to_datetime(enc_clean["ENCOUNTER_START"], utc=True, errors="coerce")
enc_clean["ENCOUNTER_STOP"]  = pd.to_datetime(enc_clean["ENCOUNTER_STOP"],  utc=True, errors="coerce")

# Keep only relevant encounter columns to keep memory usage low
enc_ref = enc_clean[["PATIENT_ID", "ENCOUNTER_ID", "ENCOUNTER_START", "ENCOUNTER_STOP", "ENCOUNTER_CLASS"]].copy()

# Prepare patients: keep relevant static columns and parse dates
patients["BIRTHDATE"] = pd.to_datetime(patients["BIRTHDATE"], errors="coerce")
patients["DEATHDATE"]  = pd.to_datetime(patients.get("DEATHDATE"), errors="coerce")  # may not exist for all

patients_ref = patients.copy()  # we'll select fields on merge time


In [20]:
# ---------- Safe group-wise merge_asof for encounters ----------
import pandas as pd
from tqdm import tqdm  # optional, for progress bar; if not installed remove tqdm usage

# Make local copies to avoid accidental mutation
left_all = patient_day_df.copy()         # has PATIENT_ID, OBS_DATE, etc.
right_all = enc_clean.copy()             # has PATIENT_ID, ENCOUNTER_ID, ENCOUNTER_START, ENCOUNTER_STOP, ENCOUNTER_CLASS

# Ensure datetime types and UTC (important)
left_all["OBS_DATE"] = pd.to_datetime(left_all["OBS_DATE"], utc=True, errors="coerce")
right_all["ENCOUNTER_START"] = pd.to_datetime(right_all["ENCOUNTER_START"], utc=True, errors="coerce")
right_all["ENCOUNTER_STOP"]  = pd.to_datetime(right_all["ENCOUNTER_STOP"],  utc=True, errors="coerce")

# Split rows with valid dates and those without
left_valid = left_all[left_all["OBS_DATE"].notna()].copy()
left_na    = left_all[left_all["OBS_DATE"].isna()].copy()  # will re-append later

# Prepare right index by patient for faster lookups
right_grouped = {pid: df.sort_values("ENCOUNTER_START").reset_index(drop=True)
                 for pid, df in right_all.groupby("PATIENT_ID")}

# We'll build a list of mapped DataFrames (one per patient)
mapped_list = []

# Iterate only patients present in left_valid to limit work
patient_ids = left_valid["PATIENT_ID"].unique()

# If tqdm isn't available remove the tqdm(...) wrapper
for pid in tqdm(patient_ids, desc="mapping encounters per patient"):
    left_grp = left_valid[left_valid["PATIENT_ID"] == pid].sort_values("OBS_DATE").reset_index(drop=True)
    right_grp = right_grouped.get(pid)

    if right_grp is None or right_grp.empty:
        # No encounters for this patient -> keep left rows, fill encounter columns with NA
        # create empty encounter columns
        left_grp = left_grp.assign(
            ENCOUNTER_ID = pd.NA,
            ENCOUNTER_START = pd.NaT,
            ENCOUNTER_STOP = pd.NaT,
            ENCOUNTER_CLASS = pd.NA
        )
        mapped_list.append(left_grp)
        continue

    # right_grp exists -> use merge_asof (left_grp and right_grp are both sorted)
    mapped = pd.merge_asof(
        left_grp,
        right_grp,
        left_on="OBS_DATE",
        right_on="ENCOUNTER_START",
        by="PATIENT_ID",
        direction="backward",
        allow_exact_matches=True
    )

    # Keep encounter info only if OBS_DATE <= ENCOUNTER_STOP (observation falls within the encounter window)
    inside_mask = mapped["ENCOUNTER_STOP"].notna() & (mapped["OBS_DATE"] <= mapped["ENCOUNTER_STOP"])
    # For rows not inside the matched encounter, clear the encounter fields
    mapped.loc[~inside_mask, ["ENCOUNTER_ID", "ENCOUNTER_START", "ENCOUNTER_STOP", "ENCOUNTER_CLASS"]] = pd.NA

    mapped_list.append(mapped)

# Concatenate mapped results
patient_day_with_enc = pd.concat(mapped_list, ignore_index=True)

# Append back the OBS_DATE NaT rows (they had no date to match against)
if not left_na.empty:
    # create encounter columns for these and set to NA
    left_na = left_na.assign(
        ENCOUNTER_ID = pd.NA,
        ENCOUNTER_START = pd.NaT,
        ENCOUNTER_STOP = pd.NaT,
        ENCOUNTER_CLASS = pd.NA
    )
    patient_day_with_enc = pd.concat([patient_day_with_enc, left_na], ignore_index=True)

# Final sort
patient_day_with_enc = patient_day_with_enc.sort_values(["PATIENT_ID", "OBS_DATE"]).reset_index(drop=True)

# Quick sanity checks
print("Rows total:", len(patient_day_with_enc))
print("Rows with encounter attached:", patient_day_with_enc["ENCOUNTER_ID"].notna().sum())
print("Sample rows with encounter info:")
print(patient_day_with_enc[patient_day_with_enc["ENCOUNTER_ID"].notna()].head(5))

# assign back to patient_day_df if you want to continue using that name
patient_day_df = patient_day_with_enc


mapping encounters per patient: 100%|██████████| 12000/12000 [03:42<00:00, 53.93it/s]


Rows total: 425050
Rows with encounter attached: 2727
Sample rows with encounter info:
                               PATIENT_ID                  OBS_DATE  DALY  \
44   00219517-ad78-f6c2-bd90-e2082558aa74 2016-05-31 20:56:04+00:00   0.0   
103  00391aa4-2858-2288-24f3-d60e0d9d0f7b 2021-03-28 10:17:13+00:00   0.0   
231  00695b1d-947e-f0c1-ad39-e0e8ee463091 2021-05-12 10:53:00+00:00   0.0   
236  006ae2f8-48fe-6d80-c64f-064bb5b9f61d 2020-12-09 12:20:13+00:00   0.0   
251  006e2c97-c4ca-c5c5-38e0-b7dee7c67ced 2025-08-10 22:55:53+00:00   0.0   

     QALY  QOLS  308136  310798  314076  59621000  \
44   49.0   1.0     NaN     NaN     NaN       NaN   
103  49.0   1.0     NaN     NaN     NaN       NaN   
231  49.0   1.0     NaN     NaN     NaN       NaN   
236   0.0   1.0     NaN     NaN     NaN       NaN   
251  49.0   1.0     NaN     NaN     NaN       NaN   

                             ENCOUNTER_ID            ENCOUNTER_START  \
44   7998a0a1-7da9-42ba-86da-ad6c720b1110  2016-05-31 20:56

In [23]:
# ---------- 4.2 Merge with conditions ----------

# Fill NaNs in conditions with 0
con_daily_wide_filled = con_daily_wide.fillna(0)

# Convert the 'DATE' column in con_daily_wide_filled to UTC to match patient_day_df['OBS_DATE']
con_daily_wide_filled['DATE'] = pd.to_datetime(con_daily_wide_filled['DATE'], utc=True)


# Merge with existing patient_day_df
patient_day_df = pd.merge(
    patient_day_df,
    con_daily_wide_filled,
    how='outer',
    left_on=['PATIENT_ID', 'OBS_DATE'],
    right_on=['PATIENT_ID', 'DATE']
)

# Drop the redundant 'DATE' column from the merge
patient_day_df = patient_day_df.drop(columns=['DATE'])

# ---------- Merge with patient demographics to get BIRTHDATE ----------
patient_day_df = pd.merge(
    patient_day_df,
    patients_subset[['PATIENT_ID', 'BIRTHDATE']],
    on='PATIENT_ID',
    how='left'
)

# Ensure date columns are in UTC before calculating age
patient_day_df["OBS_DATE"] = pd.to_datetime(patient_day_df["OBS_DATE"], utc=True, errors="coerce")
patient_day_df["BIRTHDATE"] = pd.to_datetime(patient_day_df["BIRTHDATE"], utc=True, errors="coerce")

# Recompute AGE at OBS_DATE (years) after ensuring consistent timezones
patient_day_df["AGE_AT_OBS"] = (patient_day_df["OBS_DATE"] - patient_day_df["BIRTHDATE"]).dt.days / 365.25


# Check final merged result
print("Shape after merging obs + med + con + patient demographics:", patient_day_df.shape)
print(patient_day_df.head())

Shape after merging obs + med + con + patient demographics: (936562, 17)
                             PATIENT_ID                  OBS_DATE  DALY  QALY  \
0  00092f43-1aee-3c4e-3160-58b40a45fb2b 2015-09-03 10:00:37+00:00   0.0  10.0   
1  00092f43-1aee-3c4e-3160-58b40a45fb2b 2016-09-03 10:00:37+00:00   0.0  11.0   
2  00092f43-1aee-3c4e-3160-58b40a45fb2b 2017-09-03 10:00:37+00:00   0.0  12.0   
3  00092f43-1aee-3c4e-3160-58b40a45fb2b 2018-09-03 10:00:37+00:00   0.0  13.0   
4  00092f43-1aee-3c4e-3160-58b40a45fb2b 2019-09-03 10:00:37+00:00   0.0  14.0   

   QOLS  308136  310798  314076  59621000_x ENCOUNTER_ID ENCOUNTER_START  \
0   1.0     NaN     NaN     NaN         NaN          NaN             NaT   
1   1.0     NaN     NaN     NaN         NaN          NaN             NaT   
2   1.0     NaN     NaN     NaN         NaN          NaN             NaT   
3   1.0     NaN     NaN     NaN         NaN          NaN             NaT   
4   1.0     NaN     NaN     NaN         NaN          NaN    

In [24]:
# Cell D: final cleanups

# If medication and condition columns may have NaN after merges, fill them with 0
# Find med/con columns by heuristic: numeric binary columns (you may adjust)
# Here we assume med columns were originally string codes (like '310798') and con codes too (like '59621000')
# We'll fill numeric-lookalike columns with 0 where null; but be conservative and only fill columns that are dtype float/object and contain only 0/1/NA
binary_candidates = [c for c in patient_day_df.columns if c not in ["PATIENT_ID", "OBS_DATE", "BIRTHDATE", "DEATHDATE", "ENCOUNTER_ID", "ENCOUNTER_START", "ENCOUNTER_STOP", "ENCOUNTER_CLASS", "AGE_AT_OBS"]]

# Fill NaN for candidate binary columns where appropriate
for c in binary_candidates:
    # check if column looks like a med/cond flag (values subset of {0,1,NaN} or numeric small counts)
    vals = patient_day_df[c].dropna().unique()
    if len(vals) == 0:
        continue
    # safe fill: if values are numeric and max <= 10 (cheap heuristic), fill NaN->0
    try:
        vmax = pd.to_numeric(vals, errors="coerce").max()
        if pd.notna(vmax) and vmax <= 10:
            patient_day_df[c] = patient_day_df[c].fillna(0)
    except Exception:
        # non-numeric column: skip
        pass

# If encounter columns are helpful, keep them. Otherwise drop ENCOUNTER_START/STOP if not needed:
# patient_day_df = patient_day_df.drop(columns=["ENCOUNTER_START","ENCOUNTER_STOP"])

# Sort
patient_day_df = patient_day_df.sort_values(["PATIENT_ID", "OBS_DATE"]).reset_index(drop=True)

# Final summary
print("Final patient_day_df shape:", patient_day_df.shape)
print("Rows with encounter info attached:", patient_day_df["ENCOUNTER_ID"].notna().sum())
print("Sample columns:", patient_day_df.columns.tolist()[:30])


Final patient_day_df shape: (936562, 17)
Rows with encounter info attached: 2727
Sample columns: ['PATIENT_ID', 'OBS_DATE', 'DALY', 'QALY', 'QOLS', '308136', '310798', '314076', '59621000_x', 'ENCOUNTER_ID', 'ENCOUNTER_START', 'ENCOUNTER_STOP', 'ENCOUNTER_CLASS', '59621000_y', '59621000', 'BIRTHDATE', 'AGE_AT_OBS']


In [25]:
patient_day_df

,PATIENT_ID,OBS_DATE,DALY,QALY,QOLS,308136,310798,314076,59621000_x,ENCOUNTER_ID,ENCOUNTER_START,ENCOUNTER_STOP,ENCOUNTER_CLASS,59621000_y,59621000,BIRTHDATE,AGE_AT_OBS
0,00092f43-1aee-3c4e-3160-58b40a45fb2b,2015-09-03 10:00:37+00:00,0.0,10.0,1.0,0.0,0.0,0.0,0.0,NaN,NaT,NaT,NaN,0.0,0.0,2004-09-03 00:00:00+00:00,10.997947
1,00092f43-1aee-3c4e-3160-58b40a45fb2b,2016-09-03 10:00:37+00:00,0.0,11.0,1.0,0.0,0.0,0.0,0.0,NaN,NaT,NaT,NaN,0.0,0.0,2004-09-03 00:00:00+00:00,12.000000
2,00092f43-1aee-3c4e-3160-58b40a45fb2b,2017-09-03 10:00:37+00:00,0.0,12.0,1.0,0.0,0.0,0.0,0.0,NaN,NaT,NaT,NaN,0.0,0.0,2004-09-03 00:00:00+00:00,12.999316
3,00092f43-1aee-3c4e-3160-58b40a45fb2b,2018-09-03 10:00:37+00:00,0.0,13.0,1.0,0.0,0.0,0.0,0.0,NaN,NaT,NaT,NaN,0.0,0.0,2004-09-03 00:00:00+00:00,13.998631
4,00092f43-1aee-3c4e-3160-58b40a45fb2b,2019-09-03 10:00:37+00:00,0.0,14.0,1.0,0.0,0.0,0.0,0.0,NaN,NaT,NaT,NaN,0.0,0.0,2004-09-03 00:00:00+00:00,14.997947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936557,fffc034a-0a09-cb96-1779-0953de2da5e0,2021-04-23 17:16:20+00:00,0.0,57.0,1.0,0.0,0.0,0.0,0.0,NaN,NaT,NaT,NaN,0.0,0.0,1963-04-23 00:00:00+00:00,58.001369
936558,fffc034a-0a09-cb96-1779-0953de2da5e0,2022-04-23 17:16:20+00:00,0.0,58.0,1.0,0.0,0.0,0.0,0.0,NaN,NaT,NaT,NaN,0.0,0.0,1963-04-23 00:00:00+00:00,59.000684
936559,fffc034a-0a09-cb96-1779-0953de2da5e0,2023-04-23 17:16:20+00:00,0.0,59.0,1.0,0.0,0.0,0.0,0.0,NaN,NaT,NaT,NaN,0.0,0.0,1963-04-23 00:00:00+00:00,60.000000
936560,fffc034a-0a09-cb96-1779-0953de2da5e0,2024-04-23 17:16:20+00:00,0.0,60.0,1.0,0.0,0.0,0.0,0.0,NaN,NaT,NaT,NaN,0.0,0.0,1963-04-23 00:00:00+00:00,61.002053


In [29]:
# ---------- Step 6: Finalize patient_day_df ----------

# 1) Drop duplicate condition column and rename
if "59621000_x" in patient_day_df.columns:
    patient_day_df = patient_day_df.drop(columns=["59621000_x"])
if "59621000_y" in patient_day_df.columns:
    patient_day_df = patient_day_df.rename(columns={"59621000_y": "59621000"})
# Explicitly drop any remaining duplicate '59621000' columns if they exist
patient_day_df = patient_day_df.loc[:,~patient_day_df.columns.duplicated()].copy()


# 2) Fill missing medication and condition flags with 0
flag_cols = ['308136','310798','314076','59621000']
for col in flag_cols:
    if col in patient_day_df.columns:
        patient_day_df[col] = patient_day_df[col].fillna(0)

# 3) Optional: Fill missing encounter info (if desired)
enc_cols = ['ENCOUNTER_ID','ENCOUNTER_START','ENCOUNTER_STOP','ENCOUNTER_CLASS']
for col in enc_cols:
    if col in patient_day_df.columns:
        patient_day_df[col] = patient_day_df[col].fillna("missing")

# 4) Encode categorical variables (GENDER, RACE, ETHNICITY)
cat_cols = ['GENDER','RACE','ETHNICITY']
existing_cat_cols = [c for c in cat_cols if c in patient_day_df.columns]
patient_day_df = pd.get_dummies(patient_day_df, columns=existing_cat_cols, drop_first=False)

# 5) Ensure date columns are datetime and sort
if 'OBS_DATE' in patient_day_df.columns:
    patient_day_df['OBS_DATE'] = pd.to_datetime(patient_day_df['OBS_DATE'], utc=True)
patient_day_df = patient_day_df.sort_values(["PATIENT_ID","OBS_DATE"]).reset_index(drop=True)

# 6) Final check
print("Final patient_day_df shape:", patient_day_df.shape)
print(patient_day_df.head())

Final patient_day_df shape: (936562, 36)
                             PATIENT_ID                  OBS_DATE  DALY  QALY  \
0  00092f43-1aee-3c4e-3160-58b40a45fb2b 2015-09-03 10:00:37+00:00   0.0  10.0   
1  00092f43-1aee-3c4e-3160-58b40a45fb2b 2016-09-03 10:00:37+00:00   0.0  11.0   
2  00092f43-1aee-3c4e-3160-58b40a45fb2b 2017-09-03 10:00:37+00:00   0.0  12.0   
3  00092f43-1aee-3c4e-3160-58b40a45fb2b 2018-09-03 10:00:37+00:00   0.0  13.0   
4  00092f43-1aee-3c4e-3160-58b40a45fb2b 2019-09-03 10:00:37+00:00   0.0  14.0   

   QOLS  308136  310798  314076 ENCOUNTER_ID ENCOUNTER_START  ...  \
0   1.0     0.0     0.0     0.0      missing         missing  ...   
1   1.0     0.0     0.0     0.0      missing         missing  ...   
2   1.0     0.0     0.0     0.0      missing         missing  ...   
3   1.0     0.0     0.0     0.0      missing         missing  ...   
4   1.0     0.0     0.0     0.0      missing         missing  ...   

  AGE_AT_OBS_lag_30 308136_lag_1  308136_lag_7 308136_lag

In [27]:
patient_day_df

,PATIENT_ID,OBS_DATE,DALY,QALY,QOLS,308136,310798,314076,ENCOUNTER_ID,ENCOUNTER_START,ENCOUNTER_STOP,ENCOUNTER_CLASS,59621000,59621000,BIRTHDATE,AGE_AT_OBS
0,00092f43-1aee-3c4e-3160-58b40a45fb2b,2015-09-03 10:00:37+00:00,0.0,10.0,1.0,0.0,0.0,0.0,missing,missing,missing,missing,0.0,0.0,2004-09-03 00:00:00+00:00,10.997947
1,00092f43-1aee-3c4e-3160-58b40a45fb2b,2016-09-03 10:00:37+00:00,0.0,11.0,1.0,0.0,0.0,0.0,missing,missing,missing,missing,0.0,0.0,2004-09-03 00:00:00+00:00,12.000000
2,00092f43-1aee-3c4e-3160-58b40a45fb2b,2017-09-03 10:00:37+00:00,0.0,12.0,1.0,0.0,0.0,0.0,missing,missing,missing,missing,0.0,0.0,2004-09-03 00:00:00+00:00,12.999316
3,00092f43-1aee-3c4e-3160-58b40a45fb2b,2018-09-03 10:00:37+00:00,0.0,13.0,1.0,0.0,0.0,0.0,missing,missing,missing,missing,0.0,0.0,2004-09-03 00:00:00+00:00,13.998631
4,00092f43-1aee-3c4e-3160-58b40a45fb2b,2019-09-03 10:00:37+00:00,0.0,14.0,1.0,0.0,0.0,0.0,missing,missing,missing,missing,0.0,0.0,2004-09-03 00:00:00+00:00,14.997947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936557,fffc034a-0a09-cb96-1779-0953de2da5e0,2021-04-23 17:16:20+00:00,0.0,57.0,1.0,0.0,0.0,0.0,missing,missing,missing,missing,0.0,0.0,1963-04-23 00:00:00+00:00,58.001369
936558,fffc034a-0a09-cb96-1779-0953de2da5e0,2022-04-23 17:16:20+00:00,0.0,58.0,1.0,0.0,0.0,0.0,missing,missing,missing,missing,0.0,0.0,1963-04-23 00:00:00+00:00,59.000684
936559,fffc034a-0a09-cb96-1779-0953de2da5e0,2023-04-23 17:16:20+00:00,0.0,59.0,1.0,0.0,0.0,0.0,missing,missing,missing,missing,0.0,0.0,1963-04-23 00:00:00+00:00,60.000000
936560,fffc034a-0a09-cb96-1779-0953de2da5e0,2024-04-23 17:16:20+00:00,0.0,60.0,1.0,0.0,0.0,0.0,missing,missing,missing,missing,0.0,0.0,1963-04-23 00:00:00+00:00,61.002053


In [30]:
# Define the columns for which we want lag/rolling features
numeric_obs_cols = ['DALY','QALY','QOLS','AGE_AT_OBS']  # Add other numeric columns if needed
# Update flag_cols to reflect the correct column name after dropping duplicates
flag_cols = ['308136','310798','314076','59621000']      # Meds and condition flags


all_features = numeric_obs_cols + flag_cols

# Define lag days and rolling window sizes
lag_days = [1, 7, 30]      # 1-day, 7-day, 30-day lags
rolling_windows = [7, 30]  # 7-day, 30-day rolling averages

# Group by patient
patient_groups = patient_day_df.groupby('PATIENT_ID')

# Create lag features
for col in all_features:
    for lag in lag_days:
        patient_day_df[f'{col}_lag_{lag}'] = patient_groups[col].shift(lag)

# Create rolling mean features
for col in all_features:
    for window in rolling_windows:
        patient_day_df[f'{col}_rollmean_{window}'] = patient_groups[col].shift(1).rolling(window, min_periods=1).mean()

# Optional: Fill remaining NaNs in lag/rolling with 0
patient_day_df[patient_day_df.columns.difference(['PATIENT_ID','OBS_DATE'])] = patient_day_df[patient_day_df.columns.difference(['PATIENT_ID','OBS_DATE'])].fillna(0)

# Final check
print("Shape after adding lag/rolling features:", patient_day_df.shape)
print(patient_day_df.head())

Shape after adding lag/rolling features: (936562, 55)
                             PATIENT_ID                  OBS_DATE  DALY  QALY  \
0  00092f43-1aee-3c4e-3160-58b40a45fb2b 2015-09-03 10:00:37+00:00   0.0  10.0   
1  00092f43-1aee-3c4e-3160-58b40a45fb2b 2016-09-03 10:00:37+00:00   0.0  11.0   
2  00092f43-1aee-3c4e-3160-58b40a45fb2b 2017-09-03 10:00:37+00:00   0.0  12.0   
3  00092f43-1aee-3c4e-3160-58b40a45fb2b 2018-09-03 10:00:37+00:00   0.0  13.0   
4  00092f43-1aee-3c4e-3160-58b40a45fb2b 2019-09-03 10:00:37+00:00   0.0  14.0   

   QOLS  308136  310798  314076 ENCOUNTER_ID ENCOUNTER_START  ...  \
0   1.0     0.0     0.0     0.0      missing         missing  ...   
1   1.0     0.0     0.0     0.0      missing         missing  ...   
2   1.0     0.0     0.0     0.0      missing         missing  ...   
3   1.0     0.0     0.0     0.0      missing         missing  ...   
4   1.0     0.0     0.0     0.0      missing         missing  ...   

  AGE_AT_OBS_rollmean_7 AGE_AT_OBS_rollmean_

In [31]:
# Ensure OBS_DATE is datetime
patient_day_df['OBS_DATE'] = pd.to_datetime(patient_day_df['OBS_DATE'], utc=True)

# Extract day, month, weekday
patient_day_df['day_of_week'] = patient_day_df['OBS_DATE'].dt.weekday  # Monday=0
patient_day_df['month'] = patient_day_df['OBS_DATE'].dt.month
patient_day_df['day'] = patient_day_df['OBS_DATE'].dt.day

# Optionally: weekend flag
patient_day_df['is_weekend'] = patient_day_df['day_of_week'].isin([5,6]).astype(int)

In [35]:
patient_day_df

,PATIENT_ID,OBS_DATE,DALY,QALY,QOLS,308136,310798,314076,ENCOUNTER_ID,ENCOUNTER_START,...,310798_rollmean_7,310798_rollmean_30,314076_rollmean_7,314076_rollmean_30,59621000_rollmean_7,59621000_rollmean_30,day_of_week,month,day,is_weekend
0,00092f43-1aee-3c4e-3160-58b40a45fb2b,2015-09-03 10:00:37+00:00,0.0,10.0,1.0,0.0,0.0,0.0,missing,missing,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,9.0,3.0,0
1,00092f43-1aee-3c4e-3160-58b40a45fb2b,2016-09-03 10:00:37+00:00,0.0,11.0,1.0,0.0,0.0,0.0,missing,missing,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,9.0,3.0,1
2,00092f43-1aee-3c4e-3160-58b40a45fb2b,2017-09-03 10:00:37+00:00,0.0,12.0,1.0,0.0,0.0,0.0,missing,missing,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,9.0,3.0,1
3,00092f43-1aee-3c4e-3160-58b40a45fb2b,2018-09-03 10:00:37+00:00,0.0,13.0,1.0,0.0,0.0,0.0,missing,missing,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,3.0,0
4,00092f43-1aee-3c4e-3160-58b40a45fb2b,2019-09-03 10:00:37+00:00,0.0,14.0,1.0,0.0,0.0,0.0,missing,missing,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936557,fffc034a-0a09-cb96-1779-0953de2da5e0,2021-04-23 17:16:20+00:00,0.0,57.0,1.0,0.0,0.0,0.0,missing,missing,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,23.0,0
936558,fffc034a-0a09-cb96-1779-0953de2da5e0,2022-04-23 17:16:20+00:00,0.0,58.0,1.0,0.0,0.0,0.0,missing,missing,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,23.0,1
936559,fffc034a-0a09-cb96-1779-0953de2da5e0,2023-04-23 17:16:20+00:00,0.0,59.0,1.0,0.0,0.0,0.0,missing,missing,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0,23.0,1
936560,fffc034a-0a09-cb96-1779-0953de2da5e0,2024-04-23 17:16:20+00:00,0.0,60.0,1.0,0.0,0.0,0.0,missing,missing,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,23.0,0


In [36]:
patient_day_df.columns

Index(['PATIENT_ID', 'OBS_DATE', 'DALY', 'QALY', 'QOLS', '308136', '310798',
       '314076', 'ENCOUNTER_ID', 'ENCOUNTER_START', 'ENCOUNTER_STOP',
       'ENCOUNTER_CLASS', '59621000', 'BIRTHDATE', 'AGE_AT_OBS', 'DALY_lag_1',
       'DALY_lag_7', 'DALY_lag_30', 'QALY_lag_1', 'QALY_lag_7', 'QALY_lag_30',
       'QOLS_lag_1', 'QOLS_lag_7', 'QOLS_lag_30', 'AGE_AT_OBS_lag_1',
       'AGE_AT_OBS_lag_7', 'AGE_AT_OBS_lag_30', '308136_lag_1', '308136_lag_7',
       '308136_lag_30', '310798_lag_1', '310798_lag_7', '310798_lag_30',
       '314076_lag_1', '314076_lag_7', '314076_lag_30', '59621000_lag_1',
       '59621000_lag_7', '59621000_lag_30', 'DALY_rollmean_7',
       'DALY_rollmean_30', 'QALY_rollmean_7', 'QALY_rollmean_30',
       'QOLS_rollmean_7', 'QOLS_rollmean_30', 'AGE_AT_OBS_rollmean_7',
       'AGE_AT_OBS_rollmean_30', '308136_rollmean_7', '308136_rollmean_30',
       '310798_rollmean_7', '310798_rollmean_30', '314076_rollmean_7',
       '314076_rollmean_30', '59621000_rollmean_7',

In [37]:
import pandas as pd

# -----------------------------
# 1) Forward-fill conditions/medications per patient
# -----------------------------
# List of flag columns to ffill (all numeric / binary features except core outcomes)
ffill_cols = [
    '308136', '310798', '314076', '59621000',
    '308136_lag_1','308136_lag_7','308136_lag_30',
    '310798_lag_1','310798_lag_7','310798_lag_30',
    '314076_lag_1','314076_lag_7','314076_lag_30',
    '59621000_lag_1','59621000_lag_7','59621000_lag_30',
    '308136_rollmean_7','308136_rollmean_30',
    '310798_rollmean_7','310798_rollmean_30',
    '314076_rollmean_7','314076_rollmean_30',
    '59621000_rollmean_7','59621000_rollmean_30'
]

# Forward-fill per patient with limit=7 (optional, as per previous logic)
patient_day_df[ffill_cols] = patient_day_df.groupby("PATIENT_ID")[ffill_cols].ffill(limit=7)

# -----------------------------
# 2) Handle remaining NaNs in numeric outcome features
# -----------------------------
numeric_cols = ['DALY', 'QALY', 'QOLS', 'AGE_AT_OBS',
                'DALY_lag_1','DALY_lag_7','DALY_lag_30',
                'QALY_lag_1','QALY_lag_7','QALY_lag_30',
                'QOLS_lag_1','QOLS_lag_7','QOLS_lag_30',
                'AGE_AT_OBS_lag_1','AGE_AT_OBS_lag_7','AGE_AT_OBS_lag_30',
                'DALY_rollmean_7','DALY_rollmean_30',
                'QALY_rollmean_7','QALY_rollmean_30',
                'QOLS_rollmean_7','QOLS_rollmean_30',
                'AGE_AT_OBS_rollmean_7','AGE_AT_OBS_rollmean_30']

# Fill NaNs with 0 for outcomes and age (or another imputation if needed)
patient_day_df[numeric_cols] = patient_day_df[numeric_cols].fillna(0)

# -----------------------------
# 3) Clean remaining NaNs in encounter IDs or dates
# -----------------------------
encounter_cols = ['ENCOUNTER_ID', 'ENCOUNTER_START', 'ENCOUNTER_STOP']

patient_day_df[encounter_cols] = patient_day_df[encounter_cols].fillna(pd.NA)

# -----------------------------
# 4) Final sort and index reset
# -----------------------------
patient_day_df = patient_day_df.sort_values(['PATIENT_ID', 'OBS_DATE']).reset_index(drop=True)

# -----------------------------
# 5) Optional: quick sanity checks
# -----------------------------
print("Total rows:", len(patient_day_df))
print("Rows with encounter info:", patient_day_df['ENCOUNTER_ID'].notna().sum())
print("Sample rows after cleaning:")
print(patient_day_df.head())

# -----------------------------
# Dataset is now ready for ML
# -----------------------------


Total rows: 936562
Rows with encounter info: 936562
Sample rows after cleaning:
                             PATIENT_ID                  OBS_DATE  DALY  QALY  \
0  00092f43-1aee-3c4e-3160-58b40a45fb2b 2015-09-03 10:00:37+00:00   0.0  10.0   
1  00092f43-1aee-3c4e-3160-58b40a45fb2b 2016-09-03 10:00:37+00:00   0.0  11.0   
2  00092f43-1aee-3c4e-3160-58b40a45fb2b 2017-09-03 10:00:37+00:00   0.0  12.0   
3  00092f43-1aee-3c4e-3160-58b40a45fb2b 2018-09-03 10:00:37+00:00   0.0  13.0   
4  00092f43-1aee-3c4e-3160-58b40a45fb2b 2019-09-03 10:00:37+00:00   0.0  14.0   

   QOLS  308136  310798  314076 ENCOUNTER_ID ENCOUNTER_START  ...  \
0   1.0     0.0     0.0     0.0      missing         missing  ...   
1   1.0     0.0     0.0     0.0      missing         missing  ...   
2   1.0     0.0     0.0     0.0      missing         missing  ...   
3   1.0     0.0     0.0     0.0      missing         missing  ...   
4   1.0     0.0     0.0     0.0      missing         missing  ...   

  310798_rollmean_

In [38]:
patient_day_df.dtypes

,0
PATIENT_ID,object
OBS_DATE,"datetime64[ns, UTC]"
DALY,float64
QALY,float64
QOLS,float64
308136,float64
310798,float64
314076,float64
ENCOUNTER_ID,object
ENCOUNTER_START,object


In [ ]:
import pandas as pd

# Ensure sorted by patient and time
patient_day_df = patient_day_df.sort_values(["PATIENT_ID", "OBS_DATE"])

def create_deterioration_label(patient_df):
    # Set OBS_DATE as index for rolling calculations
    patient_df = patient_df.set_index("OBS_DATE").copy()

    # Sort by index (date) for rolling
    patient_df = patient_df.sort_index(ascending=True)

    # Reverse time to use rolling for future windows
    df_rev = patient_df.sort_index(ascending=False).copy()

    # Compute rolling max/min over 90 days using the index
    df_rev["DALY_future_max"] = df_rev["DALY"].rolling("90D").max()
    df_rev["QALY_future_min"] = df_rev["QALY"].rolling("90D").min()
    df_rev["QOLS_future_min"] = df_rev["QOLS"].rolling("90D").min()

    # Convert ENCOUNTER_CLASS to boolean for rolling max
    df_rev["any_emergency"]   = (
        df_rev["ENCOUNTER_CLASS"].eq("emergency") |
        df_rev["ENCOUNTER_CLASS"].eq("inpatient")
    ).rolling("90D").max().astype(bool)


    # Flip back and reset index
    patient_df_labeled = df_rev.sort_index().reset_index()

    # Create label
    patient_df_labeled["deterioration_90d"] = (
        ((patient_df_labeled["DALY_future_max"] - patient_df_labeled["DALY"]) / patient_df_labeled["DALY"].clip(lower=1e-6) >= 0.10) |
        ((patient_df_labeled["QALY"] - patient_df_labeled["QALY_future_min"]) / patient_df_labeled["QALY"].clip(lower=1e-6) >= 0.10) |
        ((patient_df_labeled["QOLS"] - patient_df_labeled["QOLS_future_min"]) >= 0.1) |
        (patient_df_labeled["any_emergency"])
    ).astype(int)

    return patient_df_labeled

# Apply per patient
patient_day_df = patient_day_df.groupby("PATIENT_ID", group_keys=False).apply(create_deterioration_label)

# Check distribution of new label
print(patient_day_df["deterioration_90d"].value_counts(normalize=True))

In [47]:
patient_day_df['deterioration_90d'].value_counts()

,count
deterioration_90d,
0,936562
